## 拓展数据集
### 与官方数据集噪音部分结合

In [22]:
import pandas as pd
import os
import pathlib
import glob
import matplotlib. pyplot as plt
import numpy as np
import seaborn as sns
import librosa
from pydub import AudioSegment
import random
import sklearn as sk 
import scipy
from sklearn import preprocessing
from sklearn import model_selection

In [3]:
data_dir = pathlib.Path("speech_commands_v0.02")
[x for x in data_dir.iterdir() if x.is_dir()]

[WindowsPath('speech_commands_v0.02/backward'),
 WindowsPath('speech_commands_v0.02/bed'),
 WindowsPath('speech_commands_v0.02/bird'),
 WindowsPath('speech_commands_v0.02/cat'),
 WindowsPath('speech_commands_v0.02/dog'),
 WindowsPath('speech_commands_v0.02/down'),
 WindowsPath('speech_commands_v0.02/eight'),
 WindowsPath('speech_commands_v0.02/five'),
 WindowsPath('speech_commands_v0.02/follow'),
 WindowsPath('speech_commands_v0.02/forward'),
 WindowsPath('speech_commands_v0.02/four'),
 WindowsPath('speech_commands_v0.02/go'),
 WindowsPath('speech_commands_v0.02/happy'),
 WindowsPath('speech_commands_v0.02/house'),
 WindowsPath('speech_commands_v0.02/learn'),
 WindowsPath('speech_commands_v0.02/left'),
 WindowsPath('speech_commands_v0.02/marvin'),
 WindowsPath('speech_commands_v0.02/max'),
 WindowsPath('speech_commands_v0.02/nine'),
 WindowsPath('speech_commands_v0.02/no'),
 WindowsPath('speech_commands_v0.02/off'),
 WindowsPath('speech_commands_v0.02/on'),
 WindowsPath('speech_command

In [4]:
from os import listdir
from os.path import isfile, join, isdir
onlydirs = [d for d in listdir(data_dir) if isdir(join(data_dir, d))]
onlydirs

['backward',
 'bed',
 'bird',
 'cat',
 'dog',
 'down',
 'eight',
 'five',
 'follow',
 'forward',
 'four',
 'go',
 'happy',
 'house',
 'learn',
 'left',
 'marvin',
 'max',
 'nine',
 'no',
 'off',
 'on',
 'one',
 'right',
 'seven',
 'sheila',
 'six',
 'stop',
 'three',
 'tree',
 'two',
 'up',
 'visual',
 'wow',
 'yes',
 'zero',
 '_background_noise_']

In [5]:
len(onlydirs)

37

In [6]:
our_wav = [f for f in os.listdir(join(data_dir, "max")) if f.endswith(".wav")]

In [7]:
our_wav

['max_1.wav',
 'max_10.wav',
 'max_100.wav',
 'max_101.wav',
 'max_102.wav',
 'max_103.wav',
 'max_104.wav',
 'max_105.wav',
 'max_106.wav',
 'max_107.wav',
 'max_108.wav',
 'max_109.wav',
 'max_11.wav',
 'max_110.wav',
 'max_12.wav',
 'max_13.wav',
 'max_14.wav',
 'max_15.wav',
 'max_16.wav',
 'max_17.wav',
 'max_18.wav',
 'max_19.wav',
 'max_2.wav',
 'max_20.wav',
 'max_21.wav',
 'max_22.wav',
 'max_23.wav',
 'max_24.wav',
 'max_25.wav',
 'max_26.wav',
 'max_27.wav',
 'max_28.wav',
 'max_29.wav',
 'max_3.wav',
 'max_30.wav',
 'max_31.wav',
 'max_32.wav',
 'max_33.wav',
 'max_34.wav',
 'max_35.wav',
 'max_36.wav',
 'max_37.wav',
 'max_38.wav',
 'max_39.wav',
 'max_4.wav',
 'max_40.wav',
 'max_41.wav',
 'max_42.wav',
 'max_43.wav',
 'max_44.wav',
 'max_45.wav',
 'max_46.wav',
 'max_47.wav',
 'max_48.wav',
 'max_49.wav',
 'max_5.wav',
 'max_50.wav',
 'max_51.wav',
 'max_52.wav',
 'max_53.wav',
 'max_54.wav',
 'max_55.wav',
 'max_56.wav',
 'max_57.wav',
 'max_58.wav',
 'max_59.wav',
 'ma

In [16]:
our_audiosegment = []
for wav in our_wav:
    samples = AudioSegment.from_wav(join(join(data_dir, "max"), wav))
    our_audiosegment.append(samples)

In [17]:
our_audiosegment

In [18]:
background = [f for f in os.listdir(join(data_dir, "_background_noise_")) if f.endswith(".wav")]

In [19]:
background

['doing_the_dishes.wav',
 'dude_miaowing.wav',
 'exercise_bike.wav',
 'pink_noise.wav',
 'running_tap.wav',
 'white_noise.wav']

In [20]:
background_noise = []
for wav in background:
    samples = AudioSegment.from_wav(join(join(data_dir, "_background_noise_"), wav))
    background_noise.append(samples)

In [21]:
background_noise

In [24]:
SAMPLE_NUM = 5 # 每个数据随机生成5条

for our_num, _ in enumerate(our_audiosegment):
    for noise_num, _ in enumerate(background_noise):
        sound = our_audiosegment[our_num]
        for sn in range(SAMPLE_NUM):
            duration = len(our_audiosegment[our_num])
            selected_noise = background_noise[noise_num]
            start_idx = random.randint(0, len(selected_noise) - 1 - duration)
            noise = selected_noise[start_idx:(start_idx + duration)]
            # noise作为背景音的音量应为sound的一半
            noise = noise - ((1.5 * abs(sound.dBFS)) - abs(noise.dBFS))
            combined = sound.overlay(noise)
            combined.export(join(join(data_dir, "max"), f"sound{our_num}_noise{noise_num}_{sn}.wav"), format="wav")